# Testing Scheduler with different Solvers

### Import Libraries

In [3]:
import random
import test_template_from_file as tt
from ScheduleSimulator import SchedulerSimulation
import json
from copy import deepcopy

In [4]:
input_data = json.load(open("data/test_input/performanceTest1_100Requests_0.json", "r"))
sim = SchedulerSimulation(data=input_data)

In [5]:
next_events = sim.get_next_events()

In [6]:
sim.process_event_group(next_events)

In [7]:
sinfo = sim.get_scheduler_info()
sinfo

{'now': 0,
 'horizon': 604800,
 'slice_size': 300,
 'resources': {'telescope_0': [{'start': 0, 'end': 16200},
   {'start': 61200, 'end': 102600},
   {'start': 147600, 'end': 189000},
   {'start': 234000, 'end': 275400},
   {'start': 320400, 'end': 361800},
   {'start': 406800, 'end': 448200},
   {'start': 493200, 'end': 534600},
   {'start': 579600, 'end': 604800}]},
 'proposals': {'proposal_0': {'tac_priority': 25},
  'proposal_1': {'tac_priority': 20},
  'proposal_2': {'tac_priority': 25},
  'proposal_3': {'tac_priority': 30},
  'proposal_4': {'tac_priority': 25}},
 'requests': {'0': {'windows': {'telescope_0': [{'start': 91722,
      'end': 109763}]},
   'duration': 2240,
   'proposal': 'proposal_3',
   'resID': 0},
  '1': {'windows': {'telescope_0': [{'start': 66057, 'end': 71472}]},
   'duration': 600,
   'proposal': 'proposal_1',
   'resID': 1},
  '2': {'windows': {'telescope_0': [{'start': 186995, 'end': 237632}]},
   'duration': 6490,
   'proposal': 'proposal_2',
   'resID': 2}

In [8]:
sinfo.keys()

dict_keys(['now', 'horizon', 'slice_size', 'resources', 'proposals', 'requests', 'timelimit'])

## Gurobi

In [9]:
from scheduler_gurobi import SchedulerGurobi
scheduler = SchedulerGurobi(now=sinfo["now"], horizon=sinfo["horizon"], slice_size=sinfo["slice_size"],
                            resources=sinfo["resources"], proposals=sinfo["proposals"], requests=sinfo["requests"],
                            verbose=0, timelimit=sinfo["timelimit"], scheduler_type="gurobi")
results = scheduler.run()
print("Total Time: {}s".format(scheduler.get_total_time()))
print("Build: {}s, Solve: {}s, Interpret: {}s".format(scheduler.build_time, scheduler.solve_time, scheduler.interpret_time))
print("Objective Value:", scheduler.objective_value)
print("Scheduled Requests:", scheduler.scheduled_requests["scheduled"].keys())

Total Time: 0.14303350448608398s
Build: 0.09517502784729004s, Solve: 0.04381418228149414s, Interpret: 0.004044294357299805s
Objective Value: 29782.61716563539
Scheduled Requests: dict_keys(['0', '1', '6', '7', '8', '9', '11', '12', '14', '17', '21', '22', '24', '31', '34', '41', '44', '45', '48', '49', '51', '53', '54', '56', '58', '59', '62', '63', '64', '65', '67', '68', '75', '76', '80', '81', '90', '98'])


In [10]:
# Check that all Schedulers conform to this scheduler
DEFAULT_OBJ_VALUE = scheduler.objective_value
DEFAULT_SCHEDULED_REQUESTS = set(scheduler.scheduled_requests["scheduled"].keys())

## CPSAT

In [11]:
from scheduler_cpsat import SchedulerCPSAT
scheduler = SchedulerCPSAT(now=sinfo["now"], horizon=sinfo["horizon"], slice_size=sinfo["slice_size"],
                            resources=sinfo["resources"], proposals=sinfo["proposals"], requests=sinfo["requests"],
                            verbose=0, timelimit=sinfo["timelimit"], scheduler_type="cpsat")
results = scheduler.run()
print("Total Time: {}s".format(scheduler.get_total_time()))
print("Build: {}s, Solve: {}s, Interpret: {}s".format(scheduler.build_time, scheduler.solve_time, scheduler.interpret_time))
print("Objective Value Difference:", scheduler.objective_value - DEFAULT_OBJ_VALUE)
print("Same Scheduled Requests:", set(scheduler.scheduled_requests["scheduled"].keys()) == DEFAULT_SCHEDULED_REQUESTS)

Total Time: 0.31079578399658203s
Build: 0.1607069969177246s, Solve: 0.14801812171936035s, Interpret: 0.0020706653594970703s
Objective Value Difference: 3.637978807091713e-12
Same Scheduled Requests: True


## PuLP - CBC

In [12]:
from scheduler_pulp import SchedulerPulp
scheduler = SchedulerPulp(now=sinfo["now"], horizon=sinfo["horizon"], slice_size=sinfo["slice_size"],
                            resources=sinfo["resources"], proposals=sinfo["proposals"], requests=sinfo["requests"],
                            verbose=0, timelimit=sinfo["timelimit"], scheduler_type="cbc")
results = scheduler.run()
print("Total Time: {}s".format(scheduler.get_total_time()))
print("Build: {}s, Solve: {}s, Interpret: {}s".format(scheduler.build_time, scheduler.solve_time, scheduler.interpret_time))
print("Objective Value Difference:", scheduler.objective_value - DEFAULT_OBJ_VALUE)
print("Same Scheduled Requests:", set(scheduler.scheduled_requests["scheduled"].keys()) == DEFAULT_SCHEDULED_REQUESTS)

Total Time: 1.7464985847473145s
Build: 0.07606625556945801s, Solve: 1.6704323291778564s, Interpret: 0.0s
Objective Value Difference: 7.275957614183426e-12
Same Scheduled Requests: True


## PuLP - Gurobi

In [13]:
from scheduler_pulp import SchedulerPulp
scheduler = SchedulerPulp(now=sinfo["now"], horizon=sinfo["horizon"], slice_size=sinfo["slice_size"],
                            resources=sinfo["resources"], proposals=sinfo["proposals"], requests=sinfo["requests"],
                            verbose=0, timelimit=sinfo["timelimit"], scheduler_type="gurobi_pulp")
results = scheduler.run()
print("Total Time: {}s".format(scheduler.get_total_time()))
print("Build: {}s, Solve: {}s, Interpret: {}s".format(scheduler.build_time, scheduler.solve_time, scheduler.interpret_time))
print("Objective Value Difference:", scheduler.objective_value - DEFAULT_OBJ_VALUE)
print("Same Scheduled Requests:", set(scheduler.scheduled_requests["scheduled"].keys()) == DEFAULT_SCHEDULED_REQUESTS)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-20
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 505 rows, 672 columns and 6870 nonzeros
Model fingerprint: 0xd4fc7c08
Variable types: 0 continuous, 672 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 25048.687580
Presolve removed 458 rows and 570 columns
Presolve time: 0.03s
Presolved: 47 rows, 102 columns, 717 nonzeros
Found heuristic solution: objective 27940.638406
Variable types: 0 continuous, 102 integer (102 binary)
Found heuristic solution: objective 29084.846203

Root relaxation: objective 2.978262e+04, 46 iterations, 0.00 seconds (0.

## PuLP - Gurobi_CMD

In [14]:
from scheduler_pulp import SchedulerPulp
scheduler = SchedulerPulp(now=sinfo["now"], horizon=sinfo["horizon"], slice_size=sinfo["slice_size"],
                            resources=sinfo["resources"], proposals=sinfo["proposals"], requests=sinfo["requests"],
                            verbose=0, timelimit=sinfo["timelimit"], scheduler_type="gurobi_pulp_cmd")
results = scheduler.run()
print("Total Time: {}s".format(scheduler.get_total_time()))
print("Build: {}s, Solve: {}s, Interpret: {}s".format(scheduler.build_time, scheduler.solve_time, scheduler.interpret_time))
print("Objective Value Difference:", scheduler.objective_value - DEFAULT_OBJ_VALUE)
print("Same Scheduled Requests:", set(scheduler.scheduled_requests["scheduled"].keys()) == DEFAULT_SCHEDULED_REQUESTS)

Total Time: 2.486915349960327s
Build: 0.042212486267089844s, Solve: 2.4447028636932373s, Interpret: 0.0s
Objective Value Difference: 7.275957614183426e-12
Same Scheduled Requests: True


## PuLP - SCIP

In [15]:
from scheduler_pulp import SchedulerPulp
scheduler = SchedulerPulp(now=sinfo["now"], horizon=sinfo["horizon"], slice_size=sinfo["slice_size"],
                            resources=sinfo["resources"], proposals=sinfo["proposals"], requests=sinfo["requests"],
                            verbose=0, timelimit=sinfo["timelimit"], scheduler_type="scip")
results = scheduler.run()
print("Total Time: {}s".format(scheduler.get_total_time()))
print("Build: {}s, Solve: {}s, Interpret: {}s".format(scheduler.build_time, scheduler.solve_time, scheduler.interpret_time))
print("Objective Value Difference:", scheduler.objective_value - DEFAULT_OBJ_VALUE)
print("Same Scheduled Requests:", set(scheduler.scheduled_requests["scheduled"].keys()) == DEFAULT_SCHEDULED_REQUESTS)

Total Time: 0.46568751335144043s
Build: 0.036019086837768555s, Solve: 0.4246044158935547s, Interpret: 0.0050640106201171875s
Objective Value Difference: 7.275957614183426e-12
Same Scheduled Requests: True


## HiGHS

In [16]:
from scheduler_highs import SchedulerHighs
scheduler = SchedulerHighs(now=sinfo["now"], horizon=sinfo["horizon"], slice_size=sinfo["slice_size"],
                            resources=sinfo["resources"], proposals=sinfo["proposals"], requests=sinfo["requests"],
                            verbose=0, timelimit=sinfo["timelimit"], scheduler_type="highs")
results = scheduler.run()
print("Build: {}s, Solve: {}s, Interpret: {}s".format(scheduler.build_time, scheduler.solve_time, scheduler.interpret_time))
print("Total Time: {}s".format(scheduler.get_total_time()))
print("Objective Value Difference:", scheduler.objective_value - DEFAULT_OBJ_VALUE)
print("Same Scheduled Requests:", set(scheduler.scheduled_requests["scheduled"].keys()) == DEFAULT_SCHEDULED_REQUESTS)

Build: 0.06208610534667969s, Solve: 0.05869913101196289s, Interpret: 0.0s
Total Time: 0.16445708274841309s
Objective Value Difference: 0.0
Same Scheduled Requests: True
